# 🇨🇳 Chinese Learning App - MVP

A modern, interactive Chinese vocabulary learning application built for Jupyter notebooks with flashcards and matching games.

## 🚀 Key Features

### Start Screen
- **Learning Mode Selection:** Choose from 5 different modes:
  - Pinyin → Hanzi
  - Pinyin → Spanish  
  - Pinyin → English
  - Hanzi → Spanish
  - Hanzi → English
- **Word Count Selection:** Choose 3-20 words (limited by available vocabulary)
- **Two Learning Options:** Flashcards or Matching Game

### 📚 Flashcard Mode
- Modern card-based UI with gradient backgrounds
- Progress indicator showing current position
- "Show Answer" functionality
- Correct/Incorrect tracking with scoring
- Comprehensive results screen with accuracy percentage

### 🎮 Matching Game
- Grid-based card matching interface
- Visual feedback for selected and matched pairs
- Score tracking (10 points per match)
- Victory screen when all pairs are matched

## 🎨 Design Features
- **Modern UI:** Gradient backgrounds, smooth animations, rounded corners
- **Minimalistic:** Clean typography, intuitive navigation
- **Responsive:** Proper spacing and hover effects
- **Visual Feedback:** Color-coded buttons and state changes

## 📁 File Structure
The app automatically imports from `/data/sample_vocabulary.py` and includes fallback sample data if the file isn't found.

Expected vocabulary format:
```python
[
    {"hanzi": "今天", "pinyin": "jīntiān", "english": "today", "spanish": "hoy"},
    {"hanzi": "明天", "pinyin": "míngtiān", "english": "tomorrow", "spanish": "mañana"},
    # ... more entries
]
```

## 🛠 Technical Features
- **No external dependencies** beyond standard Jupyter widgets
- **Complete state management** for both game modes  
- **Error handling** for missing vocabulary file
- **Modular design** with clean separation of concerns

## 🎯 Usage Instructions
1. Run the code cell below
2. The app interface will appear
3. Select your preferred learning mode and number of words
4. Choose between Flashcards or Matching Game
5. Start learning Chinese! 

## 📝 Notes
- The app includes 10 sample words as fallback data
- Designed to be lightweight yet feature-complete
- Perfect MVP foundation for future enhancements like spaced repetition, difficulty levels, or audio pronunciation

---

**Ready to start learning? Run the cell below! ⬇️**

In [2]:
pip install ipywidgets

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.2 MB 2.9 MB/s eta 0:00:01
   ---------------------------- ----------- 1.6/2.2 MB 3.5 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import random
import json
import sys
import os


In [ ]:
# Add the data directory to Python path to import sample vocabulary
sys.path.append('/data')

try:
    from sample_vocabulary import vocabulary_data
except ImportError:
    # Fallback sample data if file doesn't exist
    vocabulary_data = [
        {"hanzi": "今天", "pinyin": "jīntiān", "english": "today", "spanish": "hoy"},
        {"hanzi": "明天", "pinyin": "míngtiān", "english": "tomorrow", "spanish": "mañana"},
        {"hanzi": "昨天", "pinyin": "zuótiān", "english": "yesterday", "spanish": "ayer"},
        {"hanzi": "水", "pinyin": "shuǐ", "english": "water", "spanish": "agua"},
        {"hanzi": "火", "pinyin": "huǒ", "english": "fire", "spanish": "fuego"},
        {"hanzi": "你好", "pinyin": "nǐ hǎo", "english": "hello", "spanish": "hola"},
        {"hanzi": "谢谢", "pinyin": "xiè xiè", "english": "thank you", "spanish": "gracias"},
        {"hanzi": "再见", "pinyin": "zài jiàn", "english": "goodbye", "spanish": "adiós"},
        {"hanzi": "学习", "pinyin": "xuéxí", "english": "to study", "spanish": "estudiar"},
        {"hanzi": "朋友", "pinyin": "péngyǒu", "english": "friend", "spanish": "amigo"}
    ]

class ChineseLearningApp:
    def __init__(self):
        self.vocabulary = vocabulary_data
        self.selected_words = []
        self.current_mode = ""
        self.num_words = 5
        self.current_card_index = 0
        self.score = 0
        self.game_pairs = []
        self.selected_cards = []
        self.matched_pairs = []
        
        # UI Components
        self.main_container = widgets.VBox()
        self.setup_styles()
        self.show_start_screen()
    
    def setup_styles(self):
        """Setup custom CSS styles"""
        display(HTML("""
        <style>
        .chinese-app {
            font-family: 'Arial', sans-serif;
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
        }
        .app-title {
            font-size: 2.5em;
            font-weight: bold;
            text-align: center;
            color: #2c3e50;
            margin-bottom: 30px;
            text-shadow: 2px 2px 4px rgba(0,0,0,0.1);
        }
        .card {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            border-radius: 15px;
            padding: 30px;
            margin: 10px;
            color: white;
            text-align: center;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
            cursor: pointer;
            transition: all 0.3s ease;
        }
        .card:hover {
            transform: translateY(-5px);
            box-shadow: 0 12px 40px rgba(0,0,0,0.15);
        }
        .hanzi {
            font-size: 3em;
            font-weight: bold;
            margin: 10px 0;
        }
        .pinyin {
            font-size: 1.5em;
            font-style: italic;
            margin: 10px 0;
        }
        .translation {
            font-size: 1.3em;
            margin: 10px 0;
        }
        .game-card {
            background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
            border-radius: 10px;
            padding: 20px;
            margin: 5px;
            color: white;
            text-align: center;
            cursor: pointer;
            min-height: 80px;
            display: flex;
            align-items: center;
            justify-content: center;
            transition: all 0.3s ease;
            box-shadow: 0 4px 15px rgba(0,0,0,0.1);
        }
        .game-card:hover {
            transform: scale(1.05);
        }
        .game-card.selected {
            background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
            transform: scale(1.1);
        }
        .game-card.matched {
            background: linear-gradient(135deg, #43e97b 0%, #38f9d7 100%);
            opacity: 0.7;
        }
        .score {
            font-size: 1.5em;
            font-weight: bold;
            color: #27ae60;
            text-align: center;
            margin: 20px 0;
        }
        .button-primary {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border: none;
            padding: 12px 30px;
            border-radius: 25px;
            font-size: 1.1em;
            font-weight: bold;
            cursor: pointer;
            transition: all 0.3s ease;
            box-shadow: 0 4px 15px rgba(0,0,0,0.2);
        }
        .button-primary:hover {
            transform: translateY(-2px);
            box-shadow: 0 6px 20px rgba(0,0,0,0.3);
        }
        </style>
        """))
    
    def show_start_screen(self):
        """Display the initial configuration screen"""
        clear_output()
        
        title = widgets.HTML("<h1 class='app-title'>🇨🇳 Chinese Learning App</h1>")
        
        # Learning mode selection
        mode_label = widgets.HTML("<h3 style='color: #2c3e50; text-align: center;'>Choose Learning Mode:</h3>")
        mode_options = [
            ("Pinyin → Hanzi", "pinyin-hanzi"),
            ("Pinyin → Spanish", "pinyin-spanish"),
            ("Pinyin → English", "pinyin-english"),
            ("Hanzi → Spanish", "hanzi-spanish"),
            ("Hanzi → English", "hanzi-english")
        ]
        
        self.mode_dropdown = widgets.Dropdown(
            options=mode_options,
            value="pinyin-hanzi",
            description="Mode:",
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px', margin='10px auto')
        )
        
        # Number of words selection
        words_label = widgets.HTML("<h3 style='color: #2c3e50; text-align: center;'>Number of Words:</h3>")
        self.words_slider = widgets.IntSlider(
            value=5,
            min=3,
            max=min(20, len(self.vocabulary)),
            step=1,
            description="Words:",
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px', margin='10px auto')
        )
        
        # Buttons
        flashcard_btn = widgets.Button(
            description="📚 Start Flashcards",
            button_style='primary',
            layout=widgets.Layout(width='200px', height='50px', margin='10px')
        )
        
        game_btn = widgets.Button(
            description="🎮 Start Matching Game",
            button_style='success',
            layout=widgets.Layout(width='200px', height='50px', margin='10px')
        )
        
        flashcard_btn.on_click(lambda x: self.start_flashcards())
        game_btn.on_click(lambda x: self.start_matching_game())
        
        buttons_box = widgets.HBox([flashcard_btn, game_btn], 
                                  layout=widgets.Layout(justify_content='center'))
        
        self.main_container.children = [
            title,
            mode_label,
            self.mode_dropdown,
            words_label,
            self.words_slider,
            widgets.HTML("<br>"),
            buttons_box
        ]
        
        display(self.main_container)
    
    def prepare_words(self):
        """Prepare the selected words based on user preferences"""
        self.current_mode = self.mode_dropdown.value
        self.num_words = self.words_slider.value
        self.selected_words = random.sample(self.vocabulary, min(self.num_words, len(self.vocabulary)))
        self.current_card_index = 0
        self.score = 0
    
    def start_flashcards(self):
        """Start flashcard learning mode"""
        self.prepare_words()
        self.show_flashcard()
    
    def show_flashcard(self):
        """Display current flashcard"""
        clear_output()
        
        if self.current_card_index >= len(self.selected_words):
            self.show_flashcard_results()
            return
        
        word = self.selected_words[self.current_card_index]
        
        # Progress indicator
        progress = widgets.HTML(f"""
            <div style='text-align: center; margin-bottom: 20px;'>
                <h2 style='color: #2c3e50;'>Flashcard {self.current_card_index + 1} of {len(self.selected_words)}</h2>
                <div style='background: #ecf0f1; height: 10px; border-radius: 5px; overflow: hidden;'>
                    <div style='background: linear-gradient(90deg, #667eea, #764ba2); height: 100%; width: {(self.current_card_index + 1) / len(self.selected_words) * 100}%; transition: width 0.3s ease;'></div>
                </div>
            </div>
        """)
        
        # Get question and answer based on mode
        question, answer = self.get_question_answer(word)
        
        # Flashcard display
        card_html = f"""
            <div class='card'>
                <div style='font-size: 1.2em; margin-bottom: 20px; opacity: 0.8;'>
                    Mode: {self.current_mode.replace('-', ' → ').title()}
                </div>
                <div style='font-size: 2.5em; margin: 20px 0;'>{question}</div>
                <div id='answer' style='display: none; font-size: 1.8em; margin-top: 30px; padding-top: 20px; border-top: 2px solid rgba(255,255,255,0.3);'>{answer}</div>
            </div>
        """
        
        card_widget = widgets.HTML(card_html)
        
        # Buttons
        show_btn = widgets.Button(description="Show Answer", button_style='info',
                                 layout=widgets.Layout(width='150px', margin='10px'))
        correct_btn = widgets.Button(description="✓ Correct", button_style='success',
                                   layout=widgets.Layout(width='150px', margin='10px'))
        incorrect_btn = widgets.Button(description="✗ Incorrect", button_style='danger',
                                     layout=widgets.Layout(width='150px', margin='10px'))
        
        show_btn.on_click(lambda x: self.show_answer())
        correct_btn.on_click(lambda x: self.answer_flashcard(True))
        incorrect_btn.on_click(lambda x: self.answer_flashcard(False))
        
        buttons_box = widgets.HBox([show_btn, correct_btn, incorrect_btn],
                                  layout=widgets.Layout(justify_content='center'))
        
        back_btn = widgets.Button(description="← Back to Menu", button_style='warning',
                                 layout=widgets.Layout(width='150px', margin='10px auto'))
        back_btn.on_click(lambda x: self.show_start_screen())
        
        self.main_container.children = [
            progress,
            card_widget,
            buttons_box,
            back_btn
        ]
        
        display(self.main_container)
    
    def get_question_answer(self, word):
        """Get question and answer based on selected mode"""
        mode_map = {
            "pinyin-hanzi": (word["pinyin"], word["hanzi"]),
            "pinyin-spanish": (word["pinyin"], word["spanish"]),
            "pinyin-english": (word["pinyin"], word["english"]),
            "hanzi-spanish": (word["hanzi"], word["spanish"]),
            "hanzi-english": (word["hanzi"], word["english"])
        }
        return mode_map.get(self.current_mode, (word["pinyin"], word["hanzi"]))
    
    def show_answer(self):
        """Show the answer on the flashcard"""
        display(HTML("""
            <script>
            document.getElementById('answer').style.display = 'block';
            </script>
        """))
    
    def answer_flashcard(self, correct):
        """Process flashcard answer and move to next"""
        if correct:
            self.score += 1
        self.current_card_index += 1
        self.show_flashcard()
    
    def show_flashcard_results(self):
        """Show flashcard session results"""
        clear_output()
        
        percentage = (self.score / len(self.selected_words)) * 100
        
        results_html = f"""
            <div style='text-align: center; padding: 40px;'>
                <h1 style='color: #2c3e50; margin-bottom: 30px;'>📊 Flashcard Results</h1>
                <div class='card' style='background: linear-gradient(135deg, #43e97b 0%, #38f9d7 100%);'>
                    <div style='font-size: 3em; margin-bottom: 20px;'>🎉</div>
                    <div style='font-size: 2em; margin-bottom: 10px;'>Score: {self.score}/{len(self.selected_words)}</div>
                    <div style='font-size: 1.5em;'>Accuracy: {percentage:.1f}%</div>
                </div>
            </div>
        """
        
        results_widget = widgets.HTML(results_html)
        
        # Buttons
        retry_btn = widgets.Button(description="🔄 Try Again", button_style='primary',
                                  layout=widgets.Layout(width='150px', margin='10px'))
        menu_btn = widgets.Button(description="🏠 Main Menu", button_style='info',
                                 layout=widgets.Layout(width='150px', margin='10px'))
        
        retry_btn.on_click(lambda x: self.start_flashcards())
        menu_btn.on_click(lambda x: self.show_start_screen())
        
        buttons_box = widgets.HBox([retry_btn, menu_btn],
                                  layout=widgets.Layout(justify_content='center'))
        
        self.main_container.children = [results_widget, buttons_box]
        display(self.main_container)
    
    def start_matching_game(self):
        """Start the matching pairs game"""
        self.prepare_words()
        self.setup_matching_game()
        self.show_matching_game()
    
    def setup_matching_game(self):
        """Setup the matching game pairs"""
        self.game_pairs = []
        self.matched_pairs = []
        self.selected_cards = []
        
        for word in self.selected_words:
            question, answer = self.get_question_answer(word)
            self.game_pairs.extend([
                {"text": question, "pair_id": len(self.game_pairs) // 2, "type": "question"},
                {"text": answer, "pair_id": len(self.game_pairs) // 2, "type": "answer"}
            ])
        
        random.shuffle(self.game_pairs)
    
    def show_matching_game(self):
        """Display the matching game"""
        clear_output()
        
        # Game header
        header = widgets.HTML(f"""
            <div style='text-align: center; margin-bottom: 20px;'>
                <h2 style='color: #2c3e50;'>🎮 Matching Game</h2>
                <div class='score'>Score: {self.score} | Pairs Found: {len(self.matched_pairs)}/{len(self.selected_words)}</div>
                <div style='color: #7f8c8d; font-size: 1.1em;'>Click two cards to match {self.current_mode.replace('-', ' with ').title()}</div>
            </div>
        """)
        
        # Create game grid
        game_buttons = []
        for i, pair in enumerate(self.game_pairs):
            btn = widgets.Button(
                description=pair["text"],
                layout=widgets.Layout(width='180px', height='80px', margin='5px'),
                button_style=''
            )
            btn.pair_info = pair
            btn.index = i
            btn.on_click(self.card_clicked)
            game_buttons.append(btn)
        
        # Arrange in grid (4 columns)
        rows = []
        for i in range(0, len(game_buttons), 4):
            row = widgets.HBox(game_buttons[i:i+4], layout=widgets.Layout(justify_content='center'))
            rows.append(row)
        
        game_grid = widgets.VBox(rows)
        
        # Back button
        back_btn = widgets.Button(description="← Back to Menu", button_style='warning',
                                 layout=widgets.Layout(width='150px', margin='20px auto'))
        back_btn.on_click(lambda x: self.show_start_screen())
        
        self.main_container.children = [header, game_grid, back_btn]
        self.game_buttons = game_buttons  # Store reference for updates
        display(self.main_container)
    
    def card_clicked(self, btn):
        """Handle card click in matching game"""
        # Ignore if card already matched or already selected
        if btn.index in self.matched_pairs or btn in self.selected_cards:
            return
        
        # Add to selected cards
        self.selected_cards.append(btn)
        btn.button_style = 'info'  # Highlight selected
        
        # Check if we have 2 selected cards
        if len(self.selected_cards) == 2:
            self.check_match()
    
    def check_match(self):
        """Check if selected cards match"""
        card1, card2 = self.selected_cards
        
        if card1.pair_info["pair_id"] == card2.pair_info["pair_id"]:
            # Match found!
            self.score += 10
            self.matched_pairs.extend([card1.index, card2.index])
            card1.button_style = 'success'
            card2.button_style = 'success'
            card1.disabled = True
            card2.disabled = True
            
            # Check if game complete
            if len(self.matched_pairs) == len(self.game_pairs):
                self.show_game_results()
        else:
            # No match
            card1.button_style = ''
            card2.button_style = ''
        
        self.selected_cards = []
        
        # Update display if game not complete
        if len(self.matched_pairs) < len(self.game_pairs):
            self.show_matching_game()
    
    def show_game_results(self):
        """Show matching game results"""
        clear_output()
        
        results_html = f"""
            <div style='text-align: center; padding: 40px;'>
                <h1 style='color: #2c3e50; margin-bottom: 30px;'>🎉 Game Complete!</h1>
                <div class='card' style='background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);'>
                    <div style='font-size: 3em; margin-bottom: 20px;'>🏆</div>
                    <div style='font-size: 2em; margin-bottom: 10px;'>Final Score: {self.score}</div>
                    <div style='font-size: 1.5em;'>All {len(self.selected_words)} pairs matched!</div>
                    <div style='font-size: 1.2em; margin-top: 15px; opacity: 0.9;'>Excellent work! 🌟</div>
                </div>
            </div>
        """
        
        results_widget = widgets.HTML(results_html)
        
        # Buttons
        play_again_btn = widgets.Button(description="🎮 Play Again", button_style='primary',
                                       layout=widgets.Layout(width='150px', margin='10px'))
        menu_btn = widgets.Button(description="🏠 Main Menu", button_style='info',
                                 layout=widgets.Layout(width='150px', margin='10px'))
        
        play_again_btn.on_click(lambda x: self.start_matching_game())
        menu_btn.on_click(lambda x: self.show_start_screen())
        
        buttons_box = widgets.HBox([play_again_btn, menu_btn],
                                  layout=widgets.Layout(justify_content='center'))
        
        self.main_container.children = [results_widget, buttons_box]
        display(self.main_container)

# Initialize and run the app
print("🚀 Starting Chinese Learning App...")
print("📚 Loading vocabulary data...")
print(f"✅ Loaded {len(vocabulary_data)} words")
print("🎨 Setting up UI...")

app = ChineseLearningApp()
print("🎉 App ready! Use the interface above to start learning.")

🎉 App ready! Use the interface above to start learning.


In [ ]:
import ipywidgets as widgets
from IPython.display import display

test_button = widgets.Button(description="Test Button")
display(test_button)